
<div align="center">
<img src="../_static/images/ugm2022/RAUGM_horizontal.png" width="90%"/>
</div> 


# Caso de uso 1: Procesamiento imagenes satelitales LANOT (Python)


## Descripción

 En este caso de estudio se utiliza la librería xarray para analizar un archivo en formato Netcdf con la información del canal 16 del satelite descargado  del Laboratorio Nacional de Observación de la Tierra (LANOT)



## Paso 1. Carga de librerías

En este caso de estudio se uilizan las siguietes librerias.

- **matplotlib** es una librería de Python especializada en la creación de gráficas estáticas, animadas e interactivas en dos dimensiones.

- **numpy** permite crear vectores y matrices multidimensionales grandes  y proporciona una gran colección de funciones matemáticas de alto nivel para operar con ellas.

- **xarray** permite etiquetar a los datos numéricos que proceden de un array de numpy lo que permite una experiencia de usuario intuitiva, concisa lo que diminuye la probabilidad de errores. 




In [1]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr


Los widgets de Jupyter  permiten convertir Jupyter Notebooks de documentos estáticos en tableros interactivos, perfectos para explorar y visualizar datos. Para hacer uso de los ipywidgets se importan los siquientes paquetes:

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## Paso 2. Lectura de archivos NetCDF

Se usa la función *open_dataset* para leer archivo netcdf del canal 16 con una resolución de 2km  24 de agosto del 2017 16:00:36 hrs

In [3]:
nc_ds =  xr.open_dataset('data/Mexico_2017.0824.160036.goes-16_C16_2km.nc')

los datos del archivo se carga en una estructura de Dataset de xarray

In [4]:
nc_ds

<xarray.Dataset>
Dimensions:      (y: 1280, x: 1920)
Dimensions without coordinates: y, x
Data variables:
    ABI-BT-ch16  (y, x) float32 ...
    latitude     (y, x) float32 ...
    longitude    (y, x) float32 ...
Attributes: (12/107)
    earth_location_notes:          Missing value for all earth location numer...
    projection_name:               LambertConformalConic
    proj_origin_latitude:          23.5
    proj_origin_longitude:         -102.0
    std_parallel_1:                18.5
    std_parallel_2:                28.5
    ...                            ...
    y\coord:                       y
    y\scale:                       2.0
    y\offset:                      0.5
    x\coord:                       x
    x\scale:                       2.0
    x\offset:                      0.5

## Paso 3. Manipulación de datos

La función *set_coords* establece a las variables latitude y longitude como  coordenadas a la estructura creada en el paso 2.

In [5]:
nc_ds.set_coords(['longitude','latitude'])

<xarray.Dataset>
Dimensions:      (y: 1280, x: 1920)
Coordinates:
    latitude     (y, x) float32 ...
    longitude    (y, x) float32 ...
Dimensions without coordinates: y, x
Data variables:
    ABI-BT-ch16  (y, x) float32 ...
Attributes: (12/107)
    earth_location_notes:          Missing value for all earth location numer...
    projection_name:               LambertConformalConic
    proj_origin_latitude:          23.5
    proj_origin_longitude:         -102.0
    std_parallel_1:                18.5
    std_parallel_2:                28.5
    ...                            ...
    y\coord:                       y
    y\scale:                       2.0
    y\offset:                      0.5
    x\coord:                       x
    x\scale:                       2.0
    x\offset:                      0.5

La propiedad *data_vars* devuelve todos los objetos de tipo *variables* que constituyen el conjunto de datos, incluidas las variables de datos y las coordenadas.

In [6]:
vars=nc_ds.data_vars
print(vars)

Data variables:
    ABI-BT-ch16  (y, x) float32 ...
    latitude     (y, x) float32 ...
    longitude    (y, x) float32 ...


*data_vars* es iterable por lo que podemos obtener una lista con los nombres de las variables de la siguiente forma:  

In [7]:
var_names=[i for i in nc_ds.data_vars]

print(var_names)


['ABI-BT-ch16', 'latitude', 'longitude']


Podemos acceder a una variable especifica indicado su nombre entre corchetes. Los atributos de la variable nos indican que los valores estan en grados kelvin.

In [8]:
variable=nc_ds['ABI-BT-ch16']
print(variable)

<xarray.DataArray 'ABI-BT-ch16' (y: 1280, x: 1920)>
[2457600 values with dtype=float32]
Dimensions without coordinates: y, x
Attributes:
    units:      temp_deg_k
    valid_min:  -3.4028235e+38
    valid_max:  3.4028235e+38


Se convierte lod valores a grados celsius restando la constante *273.15*

In [9]:
variable=variable-273.15
print(variable)

<xarray.DataArray 'ABI-BT-ch16' (y: 1280, x: 1920)>
array([[  3.0611877,   3.0760498,   3.03479  , ...,   4.61792  ,
          4.611664 ,   4.536377 ],
       [  3.0289917,   2.9838867,   2.9885254, ...,   4.765686 ,
          4.6717834,   4.5766907],
       [  3.035614 ,   2.9361877,   2.9164429, ...,   4.6710205,
          4.5852356,   4.483795 ],
       ...,
       [ -3.92984  ,  -3.226471 ,  -2.5792236, ..., -26.71521  ,
        -25.374496 , -25.022018 ],
       [ -3.6270752,  -3.1088562,  -2.5960388, ..., -28.888168 ,
        -27.607605 , -27.393661 ],
       [ -2.0161743,  -2.152649 ,  -1.6335144, ..., -31.447784 ,
        -31.349335 , -31.759903 ]], dtype=float32)
Dimensions without coordinates: y, x


Se obtienen los valores máximo y mínimo de la variable

In [10]:
val_max=variable.max().values
val_min=variable.min().values
print("valor máximo: ",val_max , "  valor mínimo: ",val_min)



valor máximo:  15.864227   valor mínimo:  -81.45831


## Paso 4. Filtrado y graficación 

Con el decorador **@interact** examina  de la función  y crea automáticamente un control con base en los tipos las entradas. En este caso se crea un control deslizante que permite elegir un número entre los valores mímnimo y máximo del conjunto de datos de la variable . El valor seleccionado se puede acceder dentro de la función ateaves de la variable x. Para este caso, el valor se usa como umbral inferior para filtrar los valores de la variable *ABI-BT-ch16*.




In [11]:
@interact
def show_articles_more_than(x=(np.round(val_min)-2,np.round(val_max))):
    
    dfint= variable.where(variable.values > x)
    
    fig = plt.figure(figsize=(6,6),dpi=200)
    dfint.plot(cmap="coolwarm",vmin=-30, vmax=45)
    

    

interactive(children=(FloatSlider(value=-33.5, description='x', max=16.0, min=-83.0), Output()), _dom_classes=…

## Referencias

- https://www.lanot.unam.mx/
- https://numpy.org/doc/
- https://docs.xarray.dev/en/stable/
- https://scitools.org.uk/cartopy/docs/latest/